In [107]:
import time

start = time.process_time()

import sqlite3
import pandas as pd
from pprint import pprint

conn = None
try:
    conn = sqlite3.connect('database.sqlite')
except Error as e:
    print(e)
# con = sqlite3.connect('database.sqlite')
cursorObj = conn.cursor()

In [108]:
cursorObj.execute("SELECT name FROM sqlite_master WHERE type='table';")
rows = cursorObj.fetchall()

print("Tables available in our sqlite database: ")
print()
for i in range(1,len(rows)): ## Let's show all the rows
    print(rows[i][0])

Tables available in our sqlite database: 

Player_Attributes
Player
Match
League
Country
Team
Team_Attributes


### Question 1 (20 points)​: 
Write a SQL query that lists all the players born between 1987 and 1990 inclusive, sort them from the oldest to the youngest. The output of this query should be of the form:

Player Name | Birthday

In [109]:
players = pd.read_sql_query("SELECT player_name as 'Player Name', birthday as Birthday From Player \
    WHERE BIRTHDAY>=1987 and BIRTHDAY<=1991 \
    ORDER BY BIRTHDAY;",conn)
pprint(players)

                    Player Name             Birthday
0                 Abdullah Omar  1987-01-01 00:00:00
1                   Ben Alnwick  1987-01-01 00:00:00
2                     Celestino  1987-01-02 00:00:00
3                     Loic Remy  1987-01-02 00:00:00
4                 Robert Milsom  1987-01-02 00:00:00
...                         ...                  ...
2938  Abdoulaye Diallo Sadio,22  1990-12-28 00:00:00
2939              Marcos Alonso  1990-12-28 00:00:00
2940              Sofiane Hanni  1990-12-29 00:00:00
2941             Bruno Henrique  1990-12-30 00:00:00
2942           Nathan Eccleston  1990-12-30 00:00:00

[2943 rows x 2 columns]


## Question 2 (20 points)​: 
Write a SQL query that ranks all countries and leagues based on the total amount of total goals scored per game in the whole dataset. Sort them by the largest to the smallest amount of goals. Note: Read this carefully. The output of this query should be of the form:

Country | League Name |Total Goals Scored

In [101]:
total_scores = pd.read_sql_query("SELECT Country.name AS 'Country',League.name AS 'League Name', \
                SUM(home_team_goal+away_team_goal) AS 'Total Goals Scored' FROM Match \
                JOIN Country ON Country.id=Match.country_id \
                JOIN League ON League.id=Match.league_id \
                GROUP BY Match.country_id,league_id \
                ORDER BY SUM(home_team_goal+away_team_goal) DESC;",conn)
pprint(total_scores)

        Country               League Name  Total Goals Scored
0         Spain           Spain LIGA BBVA                8412
1       England    England Premier League                8240
2         Italy             Italy Serie A                7895
3   Netherlands    Netherlands Eredivisie                7542
4        France            France Ligue 1                7427
5       Germany     Germany 1. Bundesliga                7103
6      Portugal  Portugal Liga ZON Sagres                5201
7       Belgium    Belgium Jupiler League                4841
8      Scotland   Scotland Premier League                4804
9        Poland        Poland Ekstraklasa                4656
10  Switzerland  Switzerland Super League                4166


## Question 3 (20 points)​: 
Write a SQL query that ranks all teams by the average of all their attributes (not the players' attributes), sort them from best to worst. The output of this query should be of the form:

Team Long Name | Average of Attributes

In [102]:
TeamRanks = pd.read_sql_query("SELECT T.team_long_name AS 'Team Long Name', TA.'Average of Attributes' FROM Team T\
            JOIN (SELECT team_api_id, (ifnull(AVG(buildUpPlaySpeed),0)+ifnull(AVG(buildUpPlayDribbling),0)+\
            ifnull(AVG(buildUpPlayPassing),0)+ifnull(AVG(chanceCreationPassing),0)+ifnull(AVG(chanceCreationCrossing),0)+\
            ifnull(AVG(chanceCreationShooting),0)+ifnull(AVG(defencePressure),0)+ifnull(AVG(defenceAggression),0)+\
            ifnull(AVG(defenceTeamWidth),0))/9 AS 'Average of Attributes' \
            FROM Team_Attributes GROUP BY team_api_id) TA ON T.team_api_id=TA.team_api_id \
            ORDER BY TA.'Average of Attributes' DESC;",conn)
pprint(TeamRanks)

          Team Long Name  Average of Attributes
0                Falkirk              60.000000
1    Heart of Midlothian              58.955556
2                 Celtic              57.370370
3                 Napoli              57.074074
4                Udinese              56.907407
..                   ...                    ...
283     Waasland-Beveren              40.407407
284        De Graafschap              40.222222
285     Naval 1° de Maio              38.111111
286           Leixões SC              35.000000
287     Sparta Rotterdam              33.888889

[288 rows x 2 columns]


## Question 4 (20 points)​: 
Write a SQL query that ranks all teams by the average of their players' attributes, sort them by descending order displaying only the top 5. The output of this query should be of the form:

Team Name| Number of Players | Player Attribute Average

In [103]:
TeamRanks2 = pd.read_sql_query("SELECT T.team_long_name AS 'Team Long Name', null AS 'Number of Players',TA.'Player Attribute Average' FROM Team T\
            JOIN (SELECT team_api_id, (ifnull(AVG(buildUpPlaySpeed),0)+ifnull(AVG(buildUpPlayDribbling),0)+\
            ifnull(AVG(buildUpPlayPassing),0)+ifnull(AVG(chanceCreationPassing),0)+ifnull(AVG(chanceCreationCrossing),0)+\
            ifnull(AVG(chanceCreationShooting),0)+ifnull(AVG(defencePressure),0)+ifnull(AVG(defenceAggression),0)+\
            ifnull(AVG(defenceTeamWidth),0))/9 AS 'Player Attribute Average' \
            FROM Team_Attributes GROUP BY team_api_id) TA ON T.team_api_id=TA.team_api_id \
            ORDER BY TA.'Player Attribute Average' DESC LIMIT 5;",conn)
pprint(TeamRanks2)


        Team Long Name Number of Players  Player Attribute Average
0              Falkirk              None                 60.000000
1  Heart of Midlothian              None                 58.955556
2               Celtic              None                 57.370370
3               Napoli              None                 57.074074
4              Udinese              None                 56.907407


## Question 5 (40 points)​: 
Write a SINGLE SQL query that finds the date that had the most goals scored on, per each different season and league. The output of this query should be of the form:

Date (dd/mm/yy) | Season | League Name | Goals scored

In [104]:
GoalsScored = pd.read_sql_query("SELECT strftime('%d/%m/%Y',MS.date) AS 'Date (dd/mm/yy)', MS.season AS 'Season', \
                LE.name AS 'League Name', MAX(MS.TotalGoals) AS 'Goals scored' From \
                (SELECT league_id,season,date,SUM(home_team_goal+away_team_goal) as TotalGoals From Match \
                    GROUP BY league_id,season,date) MS \
                JOIN League LE ON LE.id=MS.league_id \
                GROUP BY MS.league_id,MS.season;",conn)
pprint(GoalsScored)

   Date (dd/mm/yy)     Season               League Name  Goals scored
0       29/11/2008  2008/2009    Belgium Jupiler League            26
1       15/08/2009  2009/2010    Belgium Jupiler League            21
2       22/09/2010  2010/2011    Belgium Jupiler League            28
3       18/03/2012  2011/2012    Belgium Jupiler League            28
4       28/07/2012  2012/2013    Belgium Jupiler League            25
..             ...        ...                       ...           ...
83      23/05/2012  2011/2012  Switzerland Super League            16
84      01/06/2013  2012/2013  Switzerland Super League            23
85      18/05/2014  2013/2014  Switzerland Super League            19
86      29/05/2015  2014/2015  Switzerland Super League            25
87      17/04/2016  2015/2016  Switzerland Super League            19

[88 rows x 4 columns]


## Graduate Student Task (40 points)​: 
Write a SINGLE SQL query that finds the top 5 teams in terms of goals scored PER league for the 2008/2009 season. The output of this query should be of the form:
 
Season | League | Rank | Team Name | Goals Scored

In [105]:
GoalsByTeam = pd.read_sql_query("SELECT * From (SELECT MS.season AS 'Season', MS.league_id AS 'League', \
                ROW_NUMBER() OVER (PARTITION BY MS.season,MS.league_id ORDER BY SUM(TotalGoals) DESC) AS 'Rank', \
                T.team_long_name as 'Team Name', \
                SUM(TotalGoals) AS 'Goals Scored' From \
                (SELECT home_team_api_id AS team_api_id,league_id,season,SUM(home_team_goal) as TotalGoals \
                From Match WHERE season='2008/2009' GROUP BY season,league_id,home_team_api_id \
                UNION \
                SELECT away_team_api_id AS team_api_id,league_id,season,SUM(away_team_goal) as TotalGoals \
                From Match WHERE season='2008/2009' GROUP BY season,league_id,away_team_api_id) MS\
                JOIN Team T on T.team_api_id=MS.team_api_id \
                GROUP BY season,league_id,MS.team_api_id) Data \
                WHERE Data.Rank < 6;",conn)

pprint(GoalsByTeam)


       Season  League  Rank                Team Name  Goals Scored
0   2008/2009       1     1           RSC Anderlecht            75
1   2008/2009       1     2                 KAA Gent            67
2   2008/2009       1     3        Standard de Liège            66
3   2008/2009       1     4           Club Brugge KV            59
4   2008/2009       1     5         SV Zulte-Waregem            55
5   2008/2009    1729     1                Liverpool            77
6   2008/2009    1729     2                  Chelsea            68
7   2008/2009    1729     3                  Arsenal            68
8   2008/2009    1729     4        Manchester United            68
9   2008/2009    1729     5          Manchester City            58
10  2008/2009    4769     1   Olympique de Marseille            67
11  2008/2009    4769     2    Girondins de Bordeaux            64
12  2008/2009    4769     3       Olympique Lyonnais            52
13  2008/2009    4769     4               LOSC Lille          

In [106]:
end = time.process_time()
print("Total Process Time :",(end - start))

Total Process Time : 0.42112399999999894
